In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import division
import numpy as np
import scipy.signal as sg
from scipy.fftpack import rfft, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
import matplotlib.colors as colors
from pydub import AudioSegment
import math as m
import tensorflow as tf
import utilities as util

In [ ]:
bells_fs,        bells_wav,        bells_freq,        bells_time,        bells_spec,        bells_argsort,         = util.gather_data('../samples/HandBells.wav')
chords_fs,       chords_wav,       chords_freq,       chords_time,       chords_spec,       chords_argsort,        = util.gather_data('../samples/Chords.wav')
piano_mel_fs,    piano_mel_wav,    piano_mel_freq,    piano_mel_time,    piano_mel_spec,    piano_mel_argsort,     = util.gather_data('../samples/Piano.wav')
guitar_tun_fs,   guitar_tun_wav,   guitar_tun_freq,   guitar_tun_time,   guitar_tun_spec,   guitar_tun_argsort,    = util.gather_data('../samples/Guitar.wav')
piano_note_fs,   piano_note_wav,   piano_note_freq,   piano_note_time,   piano_note_spec,   piano_note_argsort,    = util.gather_data('../samples/Piano_C_Major_Note.wav')
tbone_note_fs,   tbone_note_wav,   tbone_note_freq,   tbone_note_time,   tbone_note_spec,   tbone_note_argsort,    = util.gather_data('../samples/Trombone_C_Major_Note.wav')
guitar_note_fs,  guitar_note_wav,  guitar_note_freq,  guitar_note_time,  guitar_note_spec,  guitar_note_argsort,   = util.gather_data('../samples/Guitar_C_Major_Note.wav')
piano_scale_fs,  piano_scale_wav,  piano_scale_freq,  piano_scale_time,  piano_scale_spec,  piano_scale_argsort,   = util.gather_data('../samples/Piano_C_Major_Scale.wav')
tbone_scale_fs,  tbone_scale_wav,  tbone_scale_freq,  tbone_scale_time,  tbone_scale_spec,  tbone_scale_argsort,   = util.gather_data('../samples/Trombone_C_Major_Scale.wav')
guitar_scale_fs, guitar_scale_wav, guitar_scale_freq, guitar_scale_time, guitar_scale_spec, guitar_scale_argsort,  = util.gather_data('../samples/Guitar_C_Major_Scale.wav')

In [ ]:
note_freq = util.frequencies('A0', 'D8')

In [ ]:
def spect(wav, fs):
    import numpy as np
    smallest_nps = fs/24 * 4/3 
    smallest_nps = 2**int(np.log2(smallest_nps))
    note_freq = frequencies('A0', 'D8')
    df = np.diff(note_freq)
    
    nps = np.maximum(
        np.int64(
            2**np.ceil(
                np.log2(fs/df)
            )
        ), smallest_nps
    )
    
    
    
    nps_uniq = list(np.unique(nps))
    
    
    data = []
    for nperseg in nps_uniq:
        data.append(sg.spectrogram(wav, fs, nperseg=nperseg, noverlap=nperseg-smallest_nps))
    a = 2**(1/12)
    lo_factor = 0.5*(1+1/a)
    hi_factor = 0.5*(1+a) 
    full_spec = np.zeros((len(note_freq)-1, len(data[0][1])))
    for i,(nf,nperseg) in enumerate(zip(note_freq, nps)):
        nps_ind = nps_uniq.index(nperseg)
        freq, time, spec = data[nps_ind]
        lo = nf*lo_factor
        hi = nf*hi_factor
        ind = (freq>=lo)&(freq<=hi)
#         print(nf, ind.sum(), abs(freq[ind] - nf).min() / nf)
        peak = normal_peak(spec[ind,:], freq[ind])
        full_spec[i,:len(time)] = peak

    return note_freq[:-1], data[0][1], full_spec

In [ ]:
fs = guitar_note_fs
smallest_nps = tf.truediv(fs, 18)

In [ ]:
def log2(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(2, dtype=numerator.dtype))
    return numerator / denominator

In [ ]:
new_smallest = tf.pow(2, tf.cast(log2(smallest_nps), tf.int32))

In [ ]:
def tf_diff_axis_0(a):
    return a[1:]-a[:-1]

def tf_diff_axis_1(a):
    return a[:,1:]-a[:,:-1]

In [ ]:
df = tf_diff_axis_0(note_freq)

In [ ]:
inner_log = log2(fs/df)

In [ ]:
ceil = tf.ceil(inner_log)

In [ ]:
ceil_pow = tf.pow(tf.constant(2.0, dtype='float64'), ceil)

In [ ]:
ceil_pow_64 = tf.cast(ceil_pow, tf.int64)

In [ ]:
nps = tf.maximum(ceil_pow, smallest_nps)

In [ ]:
nps

In [ ]:
nps_uniq, idx = tf.unique(nps)

In [ ]:
data = []
for nperseg in nps_uniq:
    data.append(
        sg.spectrogram(wav, fs, nperseg=nperseg, noverlap=nperseg-smallest_nps)
    )

In [ ]:
guitar_note_wav.shape

In [ ]:
wav = tf.convert_to_tensor(guitar_note_wav, dtype='float32')

In [ ]:
for u in nps_uniq:
    print(u)

for i in nps_uniq[1]:
    print(i)

In [ ]:
tf.contrib.signal.stft(wav, frame_length=nperseg, frame_step=nperseg-smallest_nps, fft_length=256)

In [ ]:
nperseg=nps_uniq[0]

In [ ]:
data = []
for nperseg in nps_uniq[1]:
    magnitude_spectrograms = tf.abs()

In [ ]:
spectrogram_patches = tf.contrib.signal.frame(magnitude_spectrograms, frame_length=64, frame_step=16, axis=1)

In [ ]:
reconstructed_signals = tf.contrib.signal.overlap_and_add(frames, frame_step=32)